# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv_file = "cities.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,rikitea,33,PF,1635121291,76,-23.1203,-134.9692,74.59,17.60
1,nikolskoye,90,RU,1635121291,77,59.7035,30.7861,35.87,8.08
2,ilulissat,81,GL,1635121292,72,69.2167,-51.1000,0.23,7.63
3,taltal,23,CL,1635121292,77,-25.4000,-70.4833,60.78,3.76
4,airai,97,TL,1635121293,51,-8.9266,125.4092,72.75,1.66


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into locations. 
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
163,dubai,0,AE,1635121189,78,25.2582,55.3047,79.05,6.91
168,riyadh,0,SA,1635121364,25,24.6877,46.7219,71.74,4.81
183,morondava,0,MG,1635121370,56,-20.2833,44.2833,72.41,4.94
255,santiago del estero,0,CL,1635121400,26,-33.4569,-70.6483,74.35,8.05
261,camargo,0,MX,1635121403,21,27.6667,-105.1667,77.86,1.43
297,jiwani,0,PK,1635121420,73,25.0500,61.7417,78.30,3.62
329,cuauhtemoc,0,MX,1635121437,21,28.4167,-106.8667,70.72,6.40
372,mecca,0,SA,1635121457,60,21.4267,39.8261,79.25,2.28
523,leticia,0,CO,1635121529,94,-4.2153,-69.9406,77.05,4.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 163: dubai.
Closest hotel is Radisson Blu Hotel, Dubai Deira Creek.
------------
Retrieving Results for Index 168: riyadh.
Closest hotel is InterContinental Riyadh, an IHG Hotel.
------------
Retrieving Results for Index 183: morondava.
Closest hotel is La Case Bambou.
------------
Retrieving Results for Index 255: santiago del estero.
Closest hotel is Che Lagarto Hostel Santiago.
------------
Retrieving Results for Index 261: camargo.
Closest hotel is Hotel Santa Fe.
------------
Retrieving Results for Index 297: jiwani.
Closest hotel is Al Mumtaz Transport Jiwani.
------------
Retrieving Results for Index 329: cuauhtemoc.
Closest hotel is Motel Tarahumara Inn.
------------
Retrieving Results for Index 372: mecca.
Closest hotel is Makkah Clock Royal Tower, A Fairmont Hotel.
------------
Retrieving Results for Index 523: leticia.
Closest hotel is Hotel Amazon Bed and Breakfast / Hotel Amazon B&B.
------------


In [6]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
163,dubai,0,AE,1635121189,78,25.2582,55.3047,79.05,6.91,"Radisson Blu Hotel, Dubai Deira Creek"
168,riyadh,0,SA,1635121364,25,24.6877,46.7219,71.74,4.81,"InterContinental Riyadh, an IHG Hotel"
183,morondava,0,MG,1635121370,56,-20.2833,44.2833,72.41,4.94,La Case Bambou
255,santiago del estero,0,CL,1635121400,26,-33.4569,-70.6483,74.35,8.05,Che Lagarto Hostel Santiago
261,camargo,0,MX,1635121403,21,27.6667,-105.1667,77.86,1.43,Hotel Santa Fe
297,jiwani,0,PK,1635121420,73,25.0500,61.7417,78.30,3.62,Al Mumtaz Transport Jiwani
329,cuauhtemoc,0,MX,1635121437,21,28.4167,-106.8667,70.72,6.40,Motel Tarahumara Inn
372,mecca,0,SA,1635121457,60,21.4267,39.8261,79.25,2.28,"Makkah Clock Royal Tower, A Fairmont Hotel"
523,leticia,0,CO,1635121529,94,-4.2153,-69.9406,77.05,4.61,Hotel Amazon Bed and Breakfast / Hotel Amazon B&B


In [7]:
# Assign the marker layer to a variable
locations = hotel_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
plt.savefig("Figures/figure13.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>